In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import os
import matplotlib.pyplot as plt
%matplotlib inline
print("PACKAGES LOADED")

PACKAGES LOADED


In [2]:
mnist = input_data.read_data_sets('data/',one_hot=True)
trainimg = mnist.train.images
trainlabel = mnist.train.labels
testimg = mnist.test.images
testlabel = mnist.test.labels
print("MNIST loaded")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
MNIST loaded


In [3]:
#DEFINE MODEL
#notwork topologies
n_input = 784
n_channel = 64#컨볼루션 필터 개수
n_classes = 10

#inputs and outputs
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

#network parameters
stddev = 0.1
weights = {
    'c1':tf.Variable(tf.random_normal([3,3,1,n_channel],stddev=stddev)),#convolution layer
    'c2':tf.Variable(tf.random_normal([3,3,n_channel,128],stddev=stddev)),
    'd1':tf.Variable(tf.random_normal([7*7*128,n_classes],stddev=stddev)),#denselayer
}
biases = {
    'c1':tf.Variable(tf.random_normal([n_channel],stddev=stddev)),
    'c2':tf.Variable(tf.random_normal([128],stddev=stddev)),
    'd1':tf.Variable(tf.random_normal([n_classes],stddev=stddev)),
}
print("NETWORK READY")


NETWORK READY


In [4]:
#MODEL
def CNN(_x, _w, _b):
    # RESHAPE(현재 벡터형태라서 텐서형태로 바꿔줘야됨.)
    _x_r = tf.reshape(_x, shape=[-1, 28, 28, 1])#앞에 몇갠지 몰라서 -1 여기선 784해줘도돼, 그다음이 28x28x1
    # CONVOLUTION
    _conv1 = tf.nn.conv2d(_x_r, _w['c1'], strides=[1, 1, 1, 1], padding='SAME')
    # ADD BIAS
    _conv2 = tf.nn.bias_add(_conv1, _b['c1'])
    # RELU
    _conv3 = tf.nn.relu(_conv2)
    #MAX-POOl
    _pool = tf.nn.max_pool(_conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
 
    # SECOND CONV LAYER
    _temp = tf.nn.conv2d(_pool, _w['c2'], strides=[1, 1, 1, 1], padding='SAME')
    _temp = tf.nn.bias_add(_temp, _b['c2'])
    _temp = tf.nn.relu(_temp)
    _temp = tf.nn.max_pool(_temp, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # VECTORIZE
    _dense = tf.reshape(_temp, [-1, _w['d1'].get_shape().as_list()[0]])#한줄로 핀다
    # DENSE
    _logit = tf.add(tf.matmul(_dense, _w['d1']), _b['d1'])
    _out = {
        'x_r' : _x_r, 'conv1' : _conv1, 'conv2' : _conv2, 'conv3' : _conv3,
        'pool' : _pool, 'dense' : _dense, 'logit' : _logit
}
    return _out

# PREDICTION
cnnout = CNN(x, weights, biases)

# LOSS AND OPTIMZER
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=cnnout['logit']))
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)#cost를 최소화하는 방향으로 아담 최적화함수를사용한다
corr = tf.equal(tf.argmax(cnnout['logit'], 1), tf.argmax(y, 1))
accr = tf.reduce_mean(tf.cast(corr, "float"))

# INITIALIZER
init = tf.global_variables_initializer()
print ("FUNCTION READY")

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

FUNCTION READY


In [5]:
#SAVER
savedir="nets/cnn_mnist_simple/"
saver=tf.train.Saver(max_to_keep=3)#??최근 3개 모델만 저장
save_step=4#얜뭐지?
if not os.path.exists(savedir):
    os.makedirs(savedir)
print("SAVER READY")

SAVER READY


In [7]:
#RUN(멀티퍼셉트론꺼 복붙)
#PARAMETERS
training_epochs=20
batch_size=100
display_step=4

#LAUNCH THE GRAPH
sess = tf.Session()
sess.run(init)

#OPTIMIZE
for epoch in range(training_epochs):
    avg_cost=0.
    total_batch=int(mnist.train.num_examples/batch_size)
    #ITERATION
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feeds={x:batch_xs, y:batch_ys}
        sess.run(optm, feed_dict=feeds)
        avg_cost+=sess.run(cost, feed_dict=feeds)
    avg_cost = avg_cost/total_batch
    
    #DISPLAY
    if(epoch+1)%display_step==0:
        print("EPOCH : %03d/%03d cost: %.9f"%(epoch,training_epochs,avg_cost))
        feeds={x:batch_xs, y:batch_ys}
        train_acc=sess.run(accr, feed_dict=feeds)
        print("TRAIN ACCURACY: %.3f" %(train_acc))
        feeds={x:mnist.test.images[:100,:], y:mnist.test.labels[:100,:]}
        test_acc=sess.run(accr,feed_dict=feeds)
        print("TEST ACCURACY: %.3f" %(test_acc))
    
    #SAVE
    if(epoch+1)%save_step==0:
        savename=savedir+"net-"+str(epoch+1)+".ckpt"
        saver.save(sess,savename)
        print("[%s] SAVED"%(savename))
print("OPTIMIZATION FINISHED")

EPOCH : 003/020 cost: 0.026428275
TRAIN ACCURACY: 0.990
TEST ACCURACY: 0.990
[nets/cnn_mnist_simple/net-4.ckpt] SAVED
EPOCH : 007/020 cost: 0.009452231
TRAIN ACCURACY: 1.000
TEST ACCURACY: 0.990
[nets/cnn_mnist_simple/net-8.ckpt] SAVED
EPOCH : 011/020 cost: 0.003263385
TRAIN ACCURACY: 1.000
TEST ACCURACY: 0.990
[nets/cnn_mnist_simple/net-12.ckpt] SAVED
EPOCH : 015/020 cost: 0.001102383
TRAIN ACCURACY: 1.000
TEST ACCURACY: 0.990
[nets/cnn_mnist_simple/net-16.ckpt] SAVED
EPOCH : 019/020 cost: 0.001045494
TRAIN ACCURACY: 1.000
TEST ACCURACY: 0.990
[nets/cnn_mnist_simple/net-20.ckpt] SAVED
OPTIMIZATION FINISHED


In [8]:
#RESTORE
do_restore = 1
if do_restore ==1:
    sess = tf.Session()
    epoch = 4
    savename = savedir+"net-"+str(epoch)+".ckpt"
    saver.restore(sess,savename)
    print("NETWORK RESTORED")
else:
    print("DO NOTHING")

ValueError: The passed save_path is not a valid checkpoint: nets/cnn_mnist_simple/net-4.ckpt